In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("WIP.owl").load()
dIDct = u.createDict(onto) 

PBNThing
BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Stakeholder_Type
Technology
ISO_Impact
ISO_Purpose
StakeholderGroup
StakeholderSubgroup
TechGroup
TechSubgroup
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
PBNCategory
RiskGroup
RiskSubgroup
CAO_Group
CAO_Subgroup
BP_Theme
BP_Category
aBlueprint


In [3]:
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


# Doing connections to Risks

In [4]:
def sparql(REQ):
    return list(default_world.sparql(REQ))

In [5]:
with onto:
    class StakeholderGroup(onto.PBNThing):
        pass
    class StakeholderSubgroup(onto.PBNThing):
        pass
    class TechGroup(onto.PBNThing):
        pass
    class TechSubgroup(onto.PBNThing):
        pass
    class RiskGroup(onto.PBNThing):
        pass
    class RiskSubgroup(onto.PBNThing):
        pass
    class has_MitigIntervention(onto.Mitigation >> onto.BP_Intervention):
        label = ["Main intervention linked to the Mitigation"]
        pass
    class has_InterventionMitig(onto.BP_Intervention >> onto.Mitigation):
        inverse_property = has_MitigIntervention
        label = ["Mitigations linked to this intervention"]
        pass


### RiskGroup -> TechGroup

In [6]:
with onto:
    class has_RiskGroup_TechGroup(onto.RiskGroup >> onto.TechGroup):
        pass
    class has_RiskGroup_TechSubgroup(onto.RiskGroup >> onto.TechSubgroup):
        pass
    class has_RiskSubgroup_TechGroup(onto.RiskSubgroup >> onto.TechGroup):
        pass
    class has_RiskSubgroup_TechSubgroup(onto.RiskSubgroup >> onto.TechSubgroup):
        pass

In [7]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechGroup.append(sTG)
        sRG.has_RiskGroup_TechGroup = list(set(sRG.has_RiskGroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
            
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechSubgroup.append(sTG)
        sRG.has_RiskGroup_TechSubgroup = list(set(sRG.has_RiskGroup_TechSubgroup))


In [8]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechGroup.append(sTG)
        sRG.has_RiskSubgroup_TechGroup = list(set(sRG.has_RiskSubgroup_TechGroup))

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techSubgroup a pbn:TechSubgroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechSubgroup.append(sTG)
        sRG.has_RiskSubgroup_TechSubgroup = list(set(sRG.has_RiskSubgroup_TechSubgroup))


### RiskGroups -> Mitigs

In [9]:
with onto:
    class has_RiskGroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        pass
    class has_RiskSubgroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        pass


In [10]:
for RG in [x.name for x in onto.RiskGroup.instances()][:-2]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskGroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskGroup a pbn:RiskGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskGroup ?riskGroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskGroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


In [11]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()][:-2]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskSubgroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskSubgroup a pbn:RiskSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskSubgroup ?riskSubgroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskSubgroup = pbn:"""+RSG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


### RiskGroups -> Owners

In [12]:
with onto:
    class has_RiskGroup_OwnerGroup(onto.RiskGroup >> onto.StakeholderGroup):
        pass
    class has_RiskGroup_OwnerSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        pass
    class has_RiskSubgroup_OwnerGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        pass
    class has_RiskSubgroup_OwnerSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        pass

In [13]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))

In [14]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))

### RiskGroups -> Subject

In [15]:
with onto:
    class has_RiskGroup_SubjectGroup(onto.RiskGroup >> onto.StakeholderGroup):
        pass
    class has_RiskGroup_SubjectSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        pass
    class has_RiskSubgroup_SubjectGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        pass
    class has_RiskSubgroup_SubjectSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        pass




In [16]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectGroup.append(sTG)
        sRG.has_RiskGroup_SubjectGroup = list(set(sRG.has_RiskGroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskGroup_SubjectSubgroup = list(set(sRG.has_RiskGroup_SubjectSubgroup))

In [17]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectGroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectGroup = list(set(sRG.has_RiskSubgroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectSubgroup = list(set(sRG.has_RiskSubgroup_SubjectSubgroup))

In [18]:
onto.save("./WIP_w_SPARQL.owl")